In [ ]:
import os

In [ ]:
I_a = preprocessing_utils.pad_for_window(I_a,
                           chopin.receptive_field_shape[0],
                           chopin.receptive_field_shape[1])

graph = graph_utils.prims_initialize(img)

ground_truth_cuts, gt_graph, gt_assignments = graph_utils.generate_gt_cuts(gt,
                                                                           seeds,
                                                                           assignments=True)

local_loss = []
local_accuracy = []

saved_models_path = os.path.join(foldername, "saved_models")

if os.path.exists(saved_models_path):
    shutil.rmtree(saved_models_path)
os.mkdir(saved_models_path)

for epoch in range(num_epochs):
    print("Epoch {}".format(epoch + 1))
    msf = chopin.predicted_msf(I_a, graph, seeds)
    segmentations = display_utils.assignments(np.zeros_like(img), msf, seeds)

    shortest_paths = nx.get_node_attributes(msf, 'path')
    assignments = nx.get_node_attributes(msf, 'seed')
    cuts = graph_utils.get_cut_edges(msf)

    acc = graph_utils.accuracy(assignments, gt_assignments)

    print("Accuracy: ", acc)
    local_accuracy.append(acc)

    filename = "epoch_{}.png".format(epoch + 1)

    boundaries = display_utils.view_boundaries(np.zeros_like(img),
                                               cuts)

    plt.imsave(os.path.join(foldername, filename), boundaries)

    constrained_msf = msf.copy()

    constrained_msf.remove_edges_from(ground_truth_cuts)

    constrained_msf = graph_utils.minimum_spanning_forest(img, constrained_msf, seeds)

    ground_truth_paths = nx.get_node_attributes(constrained_msf, 'path')

    children = graph_utils.compute_root_error_edge_children(shortest_paths,
                                                      ground_truth_paths, cuts,
                                                      ground_truth_cuts)

    weights = []
    static_images = []
    dynamic_images = []

    for (u, v), weight in children.iteritems():

        try:
            static_images.append(msf.get_edge_data(u, v)['static_image'])
            dynamic_images.append(msf.get_edge_data(u, v)['dynamic_image'])
            weights.append(weight)
            altitude_val = msf.get_edge_data(u, v)['weight']
        except KeyError:
            pass

    batches = zip(preprocessing_utils.create_batches(np.expand_dims(np.stack(weights), 1)),
                  preprocessing_utils.create_batches(np.stack(static_images)),
                  preprocessing_utils.create_batches(np.stack(dynamic_images)))

    loss = 0
    with chopin.sess.as_default():
        chopin.sess.run(chopin.zero_ops)

        for w, s, d in batches:
            feed_dict = {chopin.gradient_weights: w.transpose(),
                         chopin.static_input: s,
                         chopin.dynamic_input: d,
                         keras.backend.learning_phase(): 0}

            chopin.sess.run(chopin.accum_ops, feed_dict)
            loss = chopin.sess.run(chopin.loss, feed_dict)
            loss += loss[0][0]

        chopin.sess.run(chopin.train_step)



    local_loss.append(loss)
    print("Loss: ", loss)

    chopin.save_model("models/saved_model/Chopin/model.ckpt")
    model_name = "epoch_{}".format(epoch)
    chopin.save_model(os.path.join(foldername, "saved_models", model_name, model_name))

return segmentations, global_loss, global_accuracy